## 가우스 소거법을 이용한 일차연립방정식의 해 구하기 (row pivoting)

### pivoting이 필요한 이유 (컴퓨터에서 실수를 표현하는 방식으로 인한 rounding error)

* 컴퓨터에서 실수를 다루는 방법으로 인해 0에 가까운 수로 나눈 값은 [rounding error](https://en.wikipedia.org/wiki/Round-off_error)가 발생
* 예를 들어, 다음 연립방정식 $Ax=b$를 가우스 소거법으로 풀 때

$$[A|b]= \begin{pmatrix} \epsilon & -1 & 1&|& 0\\ -1 & 2 & -1&|& 0\\2 & -1 & 0&|& 1\end{pmatrix}$$

* $1$행 $1$열 성분 $\epsilon$을 선행성분으로 잡아 다음과 같이 가우스 소거법을 적용시키면 

$$[A'|b']=\begin{pmatrix} \epsilon & -1 & 1 &|& 0\\ 0 & 2-\frac 1{\epsilon} & -1 +\frac 1{\epsilon} &|&0\\ 0 & -1+\frac 2 {\epsilon} & -\frac 2 {\epsilon}&|&1\end{pmatrix}$$

$\epsilon$이 $0$에 가까운 값일 때, $2-\frac 1{\epsilon}$은 $-\dfrac 1{\epsilon}$으로 계산이 되므로 실제 계산되는 방정식은 

$$[A'|b']=\begin{pmatrix} \epsilon & -1 & 1 &|& 0\\ 0 & -\frac 1{\epsilon} &  \frac 1{\epsilon} &|&0\\ 0 &  \frac 2 {\epsilon} & -\frac 2 {\epsilon}&|&1\end{pmatrix}$$
이 된다.
* 이때, 원래 방정식은 해가 존재하는 방정식이지만, 계산되는 방정식은 해가 존재하지 않는 방정식이 되는 문제가 생긴다.
* 하지만, 두 행의 위치를 바꾸어(연립방정식에서 두 식의 순서를 바꾸어) 다음과 같이 풀면 이런 문제가 생기지 않는다. 

$$[A|b]=\begin{pmatrix} 2 & -1 & 0 &|& 1\\ -1 & 2 & -1 &|& 0\\ \epsilon &-1 & 1&|&0\end{pmatrix}\Longrightarrow
[A'|b']=\begin{pmatrix} 2 & -1 & 0 &|& 1\\ 0 & \frac 3 2 & -1 &|& \frac 1 2\\ 0 &-1+\frac {\epsilon}2 & 1&|&-\frac{\epsilon} 2\end{pmatrix}$$

* 위의 예에서처럼 $(1,1)$ 성분이 $2$가 되도록 행의 위치를 바꾸는 것을 row pivoting이라고 한다.


### Diagonal Dominance

* 주어진 방정식 $Ax=b$에서 $n\times n$ 행렬 $A$가 다음 조건을 만족시킬 때, $A$를 대각지배(diagonal dominance)행렬이라 한다.

$$|A_{ii}|\ge \sum_{\substack{j=1 \\ j\neq i}}^n |A_{ij}|\ (i=1,2,\cdots, n)$$

* $Ax=b$에서 $A$가 대각지배행렬이면 row pivoting이 별 영향이 없음이 알려져 있다.

### row pivoting을 이용한 가우스 소거법의 구현

* 행렬의 각 행에서 절댓값이 가장 큰 성분을 저장: $s_i = \max_{j}|A_{ij}|,\ (i=1,2,\cdots,n)$

* 가우스 소거법을 적용시킬 때, $k$번째 행에서 행의 위치를 맞바꾸어 선행성분 $A_{pk}$를 선택하는 방법:

    $r_{ij} = \dfrac{|A_{ij}|}{s_i}$ 에 대해 $r_{pk}=\max_{j}(r_{jk}),\ j\ge k\,$인 행$p$를 선택

### 가우스 소거법을 적용한 후 해를 구하는 법

* 가우스 소거법을 적용하여 $Ax=b$가 다음과 같은 형태가 되었을 때 (단, $A$는 $n\times n$행렬)
$$[A|b] = \begin{pmatrix} A_{11} & A_{12} & A_{13} & \cdots & A_{1n}&|& b_1\\
                              0  & A_{22} & A_{23} & \cdots & A_{2n}&|& b_2\\
                              0  & 0      & A_{33} & \cdots & A_{3n}&|& b_3\\
                          \vdots &\vdots  &\vdots  & \ddots & \vdots&|& \vdots\\
                              0  & 0      & 0      & 0      & A_{nn}&|& b_n\end{pmatrix}$$
                              
                              
* 해는 마지막 식에서 $x_n$을 구하는 꺼꾸로 구해나가면 되고, 이를 식으로 나타내면
$$x_n = \frac {b_n}{A_{nn}},\quad x_k = \left(b_k -\sum_{j=k+1}^n A_{kj}x_j\right) \dfrac 1 {A_{kk}},\ k=n-1,n-2,\cdots,1$$

In [1]:
import numpy as np

### numpy 배열 처리 시 주의사항

* $A = np.array([[1,2],[3,4]])$에서 $0.1*A$를 계산할 때는 $A$의 dtype이 바뀌지만
* $A[0] = 0.1*A[0]$를 할 때는 $A$의 dtype이 변하지 않음
* np.array의 일부 성분이 변할 때는 전체 dtype이 바뀌지 않음에 주의

In [2]:
A1 = np.array([[1,2],[3,4]])
A2 = A1.copy()
A1 = 0.1 * A1
A2[0] = 0.1*A2[0]
print(A1, A1.dtype)
print(A2, A2.dtype)
A2[0,0] = 4.8
print(A2)

[[0.1 0.2]
 [0.3 0.4]] float64
[[0 0]
 [3 4]] int32
[[4 0]
 [3 4]]


In [3]:
# swap

def swapRows(v,i,j):
    """swapRows(v,i,j)
       Swaps rows i and j of vector(array of rank 1) or matrix v(array of rank 2)."""
    
    if len(v.shape) == 1:
        v[i],v[j] = v[j],v[i]
    else:
        temp = v[i].copy()
        v[i] = v[j]
        v[j] = temp
        
def swapCols(v,i,j):
    """swapCols(v,i,j)
       Swaps columns i and j of matrix v(array of rank 2)."""
    
    temp = v[:,j].copy()
    v[:,j] = v[:,i]
    v[:,i] = temp

In [4]:
# gaussPivot
# numpy, swap 필요

def gaussPivot(a,b,tol=1.0e-9):
    """x = gaussPivot(a,b,tol=1.0e-9)
       Solves a x = b by Gauss elimination with scaled pivoting
       
       Both a and b are updated!
       """
    
    n = len(b)
    
    # set up scale factors
    s = np.zeros(n)
    for i in range(n):
        s[i] = np.max(np.abs(a[i,:]))
        
    for k in range(0,n-1):
        # 필요할 때 행교환
        p = np.argmax(np.abs(a[k:n,k])/s[k:n]) + k
        if np.abs(a[p,k]) < tol:
            print("가역행렬이 아닙니다.")
            return None
        if p != k:
            swapRows(b,k,p)    # swap을 모듈로 저장시 swap.swapRows
            swapRows(s,k,p)
            swapRows(a,k,p)
            
        # 가우스 소거법
        for i in range(k+1,n):
            if a[i,k] != 0.0:
                lamb = a[i,k]/a[k,k]
                a[i,k+1:n] = a[i,k+1:n] - lamb * a[k,k+1:n]
                b[i] = b[i] - lamb * b[k]
    if np.abs(a[n-1,n-1]) < tol:
        print("가역행렬이 아닙니다.")
        return None 
    
    for k in range(n-1,-1,-1):
        b[k] = (b[k] - np.dot(a[k,k+1:n], b[k+1:n])) / a[k,k]
    return b

In [5]:
A = np.array([[2,-2,6],[-2,4,3],[-1,8,4]])
b = np.array([16,0,-1])

In [6]:
A1 = 1. * A.copy()
b1 = 1. * b.copy()

In [7]:
x = gaussPivot(A1, b1)

In [8]:
x

array([ 1., -1.,  2.])

In [9]:
np.dot(A, x)

array([16.,  0., -1.])

In [10]:
A2 = A.copy()
b2 = b.copy()
x2 = gaussPivot(A2 * 1.0,b2 * 1.0)

In [11]:
x2

array([ 1., -1.,  2.])

In [12]:
np.dot(A,x2)

array([16.,  0., -1.])

## $n\times n$ 행렬의 LU 분해

### row pivoting이 없는 가우스 소거법과 LU 분해

* $n\times n$ 행렬 $A$를 하삼각행렬 $L$과 상삼각행렬 $U$의 곱 $A = LU$로 나타내는 것을 LU 분해라고 함

* 행렬 $A$의 LU 분해는 일반적으로 유일하지 않지만, 특별한 조건하에서는 $L$과 $U$가 유일하게 결정됨 (아래 예 참고)
    - 하삼각행렬 $L$의 대각성분이 모두 $1$인 경우 : Doolittle 분해
    - 상삼각행렬 $U$의 대각성분이 모두 $1$인 경우 : Crout 분해
    - 행렬 $A$가 양의 준정부호 행렬일 때, $L=U^{\rm T}$인 경우 : Choleski 분해
    
* 예를 들어, $3 \times 3$인 경우에 $L$의 대각성분이 모두 $1$이 되도록 분해하는 경우

$$L = \begin{pmatrix} 1 & 0 & 0\\L_{21} & 1 & 0\\ L_{31} & L_{32} & 1\end{pmatrix},\ 
U=\begin{pmatrix} U_{11} & U_{12} & U_{13}\\ 0 & U_{22} & U_{23}\\ 0 & 0 & U_{33}\end{pmatrix},\\
A = LU = \begin{pmatrix} U_{11} & U_{12} & U_{13}\\ U_{11}L_{21} & U_{12}L_{21}+U_{22}& U_{13}L_{21}+U_{23}\\
U_{11}L_{31} & U_{12}L_{31}+U_{22}L_{32} & U_{13}L_{31}+U_{23}L_{32}+U_{33}\end{pmatrix}$$

* 가우스 소거법을 적용시켜보면, 가우스 소거법에서 한 행의 상수배를 다른 행에서 뺄 때 곱하는 상수값이 $L$의 성분이고, 가우스 소거법의 결과로 얻어지는 행렬이 $U$가 됨


### 행렬의 LU 분해를 알 때, 연립방정식의 해를 구하는 법

* $Ax = b \Longleftrightarrow LUx=b$ 

* $Ux = y$로 두고, $Ly = b$를 $y_1$부터 $y_n$까지 순차적으로 구하여 $y=(y_1,\cdots,y_n)^{\rm T}$를 계산

$$y_1 = b_1,\quad y_k = b_k -\sum_{j=1}^{k-1} L_{kj}y_j,\ (k=2,3,\cdots,n)$$

* 계산된 $y$를 이용하여 $Ux=y$를 $x_n$부터 $x_1$까지 순차적으로 구하여 $x$를 계산 : 계산 방법은 가우스 소거법에서 처럼

### row pivoting이 있는 가우스 소거법과 LU분해 (Doolittle 분해)

* 가우스 소거법에서 두 행의 위치를 맞바꾸는 swap이 일어나는 경우 이 정보를 담고 있는 치환행렬 $P$와 하삼각행렬 $L$ 그리고 상상각행렬 $U$에 대해 $PA = LU$와 같이 분해할 수 있다. 

* 아래 구현에서 하삼각행렬의 대각성분이 모두 1인 LU 분해(Doolittle 분해)를 할 것이므로, 입렬 행렬 $A$를 업데이트하여 $[L\setminus U]$꼴로 반환하고, swap이 일어난 정보를 seq에 담아서 반환하도록 구현

$$[L\setminus U] = \begin{pmatrix} U_{11} & U_{12} & U_{13} & \cdots & U_{1n}\\
L_{21} & U_{22} & U_{23} & \cdots & U_{2n}\\ L_{31} & L_{32} & U_{33} & \cdots & U_{3n}\\
\vdots & \vdots & \vdots &\ddots & \vdots & \\ L_{n1} & L_{n2} & L_{n3} & \cdots & U_{nn}\end{pmatrix}$$

In [13]:
def LUdecomp(a, tol=1.0e-9):
    """a, seq = LUdecomp(a,tol=1.0e-9)
       * returned matrix [a] is of the form [L/U] 
       * [L][U] is a row-wise permutation of the original matrix [a] 
         and the permutations are recorded in teh vector [seq]"""
    
    n = len(a)
    seq = np.array(range(n))
    
    # set up scale factors
    s = np.zeros((n), dtype = float)
    for i in range(n):
        s[i] = np.max(np.abs(a[i,:]))
        
    for k in range(0,n-1):
        # 필요할 때 행 교환(swap)
        p = int(np.argmax(np.abs(a[k:n, k]) / s[k:n])) + k
        if np.abs(a[p,k]) < tol:
            print("가역행렬이 아닙니다.")
        if p != k:
            swapRows(s,k,p)
            swapRows(a,k,p)
            swapRows(seq,k,p)
            
        # 가우스 소거법
        for i in range(k+1,n):
            if a[i,k] != 0.0:
                lamb = a[i,k]/a[k,k]
                a[i, k+1:n] = a[i, k+1:n] - lamb * a[k, k+1:n]
                a[i,k] = lamb   ### 가우스 소거법과 다른 부분 (하삼각행렬 계산)
    return a, seq

In [14]:
def LUsolve_pre(a,b,seq):
    """x = LUsolve(a,b,seq)
       [L][U]x = b를 만족하는 x를 구하는 함수
       [a]=[L/U]와 seq는 a, seq = LUdecomp(a)로 얻어진 값"""
    
    n = len(a)
    
    # seq를 이용하여 b를 재정렬하기
    x = b.copy()
    for i in range(n):
        x[i] = b[seq[i]]
        
    # 해를 구하기 (Ly = b인 y 구하기)
    for k in range(1,n):
        x[k] = x[k] - np.dot(a[k, 0:k], x[0:k])
        
    # 해를 구하기 (Ux = y인 x 구하기)
    for k in range(n-1,-1,-1):
        x[k] = (x[k] - np.dot(a[k, k+1:n], x[k+1:n])) / a[k,k]
    return x

In [15]:
def LUsolve(a,b,tol=1.0e-9):
    """x = LUsolve(a,b,tol=1.0e-9)
       [a][x]=[b]를 만족하는 해 x를 LU 분해를 이용하여 구하는 함수"""
    
    _, seq = LUdecomp(a,tol)
    return LUsolve_pre(a,b,seq)

In [16]:
A = 1.0 * np.array([[2,-2,6],[-2,4,3],[-1,8,4]])
b = 1.0 * np.array([16,0,-1])

In [17]:
a, seq = LUdecomp(A)

In [18]:
x = LUsolve_pre(a, b, seq)

In [19]:
x

array([ 1., -1.,  2.])

In [20]:
A = 1.0 * np.array([[2,-2,6],[-2,4,3],[-1,8,4]])
b = 1.0 * np.array([16,0,-1])

LUsolve(A,b)

array([ 1., -1.,  2.])

In [21]:
import sys
sys.path

['C:\\Users\\Kil-Chan Ha\\Documents\\Cal2020',
 'C:\\Users\\Kil-Chan Ha\\.conda\\envs\\Cal2020\\python37.zip',
 'C:\\Users\\Kil-Chan Ha\\.conda\\envs\\Cal2020\\DLLs',
 'C:\\Users\\Kil-Chan Ha\\.conda\\envs\\Cal2020\\lib',
 'C:\\Users\\Kil-Chan Ha\\.conda\\envs\\Cal2020',
 '',
 'C:\\Users\\Kil-Chan Ha\\.conda\\envs\\Cal2020\\lib\\site-packages',
 'C:\\Users\\Kil-Chan Ha\\.conda\\envs\\Cal2020\\lib\\site-packages\\win32',
 'C:\\Users\\Kil-Chan Ha\\.conda\\envs\\Cal2020\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\Kil-Chan Ha\\.conda\\envs\\Cal2020\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\Kil-Chan Ha\\.conda\\envs\\Cal2020\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Kil-Chan Ha\\.ipython']